# Laboratorio 2 - Pregunta 2
### Hecho por: Daniel Alonso, Alvaro García Cid, Enrique Ramos e Ignacio Regidor

En esta práctica, se estudiará la secuencia de genes asociada a distintas clases, **EI** (empalme exon-intron), **IE** (empalme intron-exon) y **N** (muestra sin empalme). 

A partir de la base de datos utilizada para el estudio, se intentará encontrar patrones secuenciales (porque el orden es importante) para estudiarlos posteriormente. 


##Introducción al problema y pasos previos
A continuación se muestrán las librerías de Python utilizadas en esta práctica.
- Numpy: utilizada por el resto de liberías, permite la creación y el manejo de datos de forma rápida y eficiente, conteniendo estructuras de datos propias.
- Pandas: permite la creación y manejo de dataframes a partir de la base de datos empleada.
- Matplotlib.pyplot: se utilizará para graficar los distintos diagramas que permitan el estudio de los datos de forma visual.
- gsppy.gsp: se utilizará el módulo GSP de esta librería, que permitirá aplicar el algoritmo Generalized Sequential Patterns.
- random: se utilizará para crear números aleatorios que determinarán la longitud de las subsecuencias creadas a partir de la secuencia de genes de cada muestra.

In [ ]:
!pip install gsppy
from gsppy.gsp import GSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Se fija una semilla para que los números aleatorios usados posteriormente sean 
# siempre los mismos cada vez que se ejecuta el código.
random.seed(123) 

In [ ]:
datosraw = pd.read_csv("splice.data", encoding='latin-1', sep=',', names = ['Clase', 'Donante', 'Secuencias'])
datosraw.head()
datosraw.shape

(3190, 3)

La base de datos sin preprocesar cuenta con 3190 muestras cada una con 3 atributos.

In [ ]:
datosraw.head()

,Clase,Donante,Secuencias
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGG...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGG...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTG...


Los atributos de cada muestra son la clase, el donante y la secuencia de genes.

##Preprocesamiento de los datos

In [ ]:
print("Duplicated Values:", datosraw.duplicated().sum())

Duplicated Values: 12


Hay 12 filas del dataframe duplicadas por lo que se procede a eliminarlas.

Al ser datos del mismo donante, clase y secuencia de genes, al realizar el análisis de asociación, estas filas tendrían más peso y relevancia solo por el hecho de ser información duplicada de la misma muestra; y no porque estos datos se repitan en otras muestras diferentes.

In [ ]:
datosraw = datosraw.drop_duplicates()

In [ ]:
print("Duplicated Values:", datosraw.duplicated().sum())

Duplicated Values: 0


In [ ]:
for c in datosraw.columns:
  print("Missing Values [{0}]:".format(c), datosraw[c].isna().sum())

Missing Values [Clase]: 0
Missing Values [Donante]: 0
Missing Values [Secuencias]: 0


Como no hay valores nulos, ya se ha completado la limpieza de datos. Como el estudio se basa en encontrar patrones secuenciales (que serán caracteres) en los distintos tipos de enlace, no tiene sentido buscar outliers.

El siguiente paso del preprocesamiento de los datos, es eliminar los caracteres vacíos o de espacio de la secuencia de genes. Además, se crea una nueva columna llamada **Split**. De cada fila se cogerá aleatoriamente entre 4 y 10 genes (que serán los primeros de la columna **Secuencia**). Estos, se asignarán al valor en la fila de la nueva columna creada.

In [ ]:
pd.set_option('mode.chained_assignment', None) #Suprimimos warning por hacer copia
datos = datosraw.copy().reset_index()
datos['Split'] = 0  
#Eliminamos carácteres vacíos que hay antes de la secuencia de genes
for i in range(len(datos)):
  aux = ''
  for j in range(len(datos['Secuencias'][i])):
    if datos['Secuencias'][i][j] != ' ':
      aux = aux + datos['Secuencias'][i][j]
  datos['Secuencias'][i] = aux

#Hacemos partición aleatoria de los primeros n (entre 4 y 10) genes de Secuencias
for i in range(len(datos)):
  n = random.randint(4,10)
  aux = ''
  for j in range(n):
    aux = aux + datos['Secuencias'][i][j]
  datos['Split'][i] = aux

In [ ]:
datos.head()

,index,Clase,Donante,Secuencias,Split
0,0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...,CCAG
1,1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...,AGACCC
2,2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...,GAGG
3,3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...,GGGCTGCGTT
4,4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...,GCTCAGC


El dataframe parece que ha quedado correctamente, pero por si acaso se comprueba que la columna Split contenga secuencias de entre 4 y 10 genes.

In [ ]:
#Comprobamos que la partición este bien hecha
comp = 0
for i in range(len(datos)):
  if len(datos['Split'][i]) >= 4 & len(datos['Split'][i]) <= 10:
    comp = comp + 1

if comp == len(datos):
  print("Todos tienen una longitud entre 4 y 10")
else:
  print("Hay algún error")

Todos tienen una longitud entre 4 y 10


A continuación, se muestran las clases presentes en el dataframe, y si existe algún donante al que le corresponda más de una muestra. 

Esto, haría que los datos estuviesen sesgados, al tener mayor peso la genética de un donante sobre las demás, aunque fueran diferentes secuencias.

In [ ]:
print('Las clases son: ', datos['Clase'].unique()) #Diferentes clases
print('El número de donantes con más de una muestra es: ', len(datos['Donante']) - len(datos['Donante'].unique()))

Las clases son:  ['EI' 'IE' 'N']
El número de donantes con más de una muestra es:  0


Se observa que a cada donante le corresponde una única muestra. 

Sin embargo, se debe estudiar si dentro de la nueva columna **Split**, existen secuencias exactamente iguales que pertenezcan a la misma clase. 

Esto, se realiza porque el objetivo no es encontrar patrones de secuencias, sino de subsecuencias contenidas en estas. 

Si se incluyen secuencias repetidas en el estudio dentro de una misma clase, la frecuencia de aparición de ciertas subsecuencias puede parecer mayor solo por aparecer en un solo tipo de secuencia (que sea siempre igual), aunque apenas aparezca en las demás.

In [ ]:
datos.duplicated(subset=['Clase','Split']).any()

True

Se procede a eliminar las filas duplicadas simultáneamente en los valores de **Clase** y **Split**.

In [ ]:
datos.drop_duplicates(subset=['Clase','Split'], inplace=True)

## Resolución del problema

A continuación, se utiliza la librería GSP para encontrar patrones secuenciales. Como no se dispone de mucha información en internet sobre esta libería, ni por parte del creador ni de otros usuarios, se procede a realizar varias pruebas con ella para entender como funciona.

Para ello, se van a crear varios strings de prueba para estudiar que patrones secuenciales es capaz de encontrar la librería.

In [ ]:
pruebas1 = [['a','b','c'],['a','b','d'],['a','b','e'],['a','b','f'],['a','b','g'],['a','b','h']]
results_pruebas1 = GSP(pruebas1).search(0.2)
# 0.2 hace referencia al soporte mínimo para ser frecuente.
print(results_pruebas1)

[{('b',): 6, ('a',): 6}, {('a', 'b'): 6}]


In [ ]:
pruebas2 = ['abc','abd','abe','abf','abg','abh']
results_pruebas2 = GSP(pruebas2).search(0.2)
print(results_pruebas2)

[{('a',): 6, ('b',): 6}, {('a', 'b'): 6}]


A partir de las pruebas 1 y 2, se observa que en ambos casos es capaz de reconocer por igual cuando el caracter 'a' va seguido del caracter 'b'.

Esto, quiere decir que la librería va a obtener los mismos resultados, independientemente de si cada muestra de secuencia de genes se introduce como un bloque de caracteres (que es como están actualmente en el dataframe en forma de string) o si se introducen separados individualmente los caracteres.

In [ ]:
pruebas3 = [['b','d','a'],['b','e','a'],['b','f','a'],['b','g','a'],['b','h','a'],['b','i','a']]
results_pruebas3 = GSP(pruebas3).search(0.2)
print(results_pruebas3)

[{('b',): 6, ('a',): 6}]


A partir de la prueba 3, se puede observar que la librería no es capaz de reconocer que 'a' y 'b' están en todos los strings introducidos habiendo una separación entre ellos de un caracter. Es decir, la librería solo detecta patrones de 2 o más caracteres que estén inmediatamente seguidos.

Además, como todos los strings acaban con el caracter 'b' y empiezan por 'a', se puede deducir que la librería tampoco detecta patrones entre distintos strings aunque estén inmediatamente seguidos. Esto, es bueno para este estudio, ya que cada muestra es independiente de las demás.

Una vez entendido como funciona la librería, se utiliza con el dataframe para encontrar patrones secuenciales en la columna **Split**. Se buscarán patrones generales, y luego caraterísticos de cada clase (EI, IE y N).

Se utilizarán distintos soportes. El primero de ellos es **0.2**.

### GSP con soporte 0.2

In [ ]:
results = GSP(datos['Split']).search(0.2)
resultEI = GSP(datos['Split'][datos['Clase']=='EI']).search(0.2)
resultIE = GSP(datos['Split'][datos['Clase']=='IE']).search(0.2)
resultN = GSP(datos['Split'][datos['Clase']=='N']).search(0.2)

In [ ]:
print('Patrones secuenciales generales:\n',results)
print('\n\nPatrones secuenciales clase EI:\n',resultEI)
print('\n\nPatrones secuenciales clase IE:\n',resultIE)
print('\n\nPatrones secuenciales clase N:\n',resultN)

Patrones secuenciales generales:
 [{('C',): 2424, ('A',): 2351, ('G',): 2370, ('T',): 2346}, {('A', 'C'): 863, ('A', 'A'): 781, ('A', 'G'): 1036, ('A', 'T'): 800, ('C', 'A'): 1084, ('C', 'C'): 996, ('C', 'T'): 1271, ('G', 'A'): 1024, ('G', 'G'): 921, ('G', 'C'): 1041, ('G', 'T'): 675, ('T', 'C'): 965, ('T', 'T'): 745, ('T', 'G'): 1166}]


Patrones secuenciales clase EI:
 [{('C',): 579, ('A',): 534, ('T',): 541, ('G',): 572}, {('A', 'C'): 192, ('A', 'A'): 162, ('A', 'G'): 259, ('A', 'T'): 173, ('C', 'A'): 238, ('C', 'C'): 229, ('C', 'G'): 144, ('C', 'T'): 294, ('G', 'A'): 269, ('G', 'C'): 281, ('G', 'G'): 242, ('T', 'C'): 217, ('T', 'G'): 282, ('T', 'T'): 142}]


Patrones secuenciales clase IE:
 [{('C',): 614, ('T',): 595, ('A',): 548, ('G',): 525}, {('A', 'A'): 186, ('A', 'C'): 237, ('A', 'T'): 184, ('A', 'G'): 186, ('C', 'A'): 268, ('C', 'C'): 284, ('C', 'T'): 370, ('G', 'A'): 192, ('G', 'C'): 224, ('G', 'G'): 160, ('G', 'T'): 173, ('T', 'C'): 285, ('T', 'T'): 193, ('T', 'G'): 277}, {

A partir de estos datos, se puede determinar cual es la frecuencia de soporte de cada caracter suelto. Estos caracteres, aparecen ya que por la regla antimonótona si lo hacen sus superconjuntos, también deben ser frecuentes por separado. A partir de estos datos, se puede determinar que el dataset está correctamente equilibrado en cuanto al número de caracteres de cada tipo que hay.

Además, se pueden observar combinaciones de 2 caracteres ordenados frecuentes, y si son frecuentes en todas las clases o para una en concreto.
 
Por otro lado, solo aparece la combinación 'CCT' en la clase **IE**, como patrón secuencial frecuente de longitud mayor que 2 caracteres (3 caracteres). A partir de esto, y que se ha utilizado un soporte mínimo relativamente bajo (0.2), se puede deducir que los patrones existentes normalmente solo pueden ser de longitud 2 y que puede estudiarse también la presencia de cada caracter individualmente en cada clase o en general.

Patrones interesantes:
- **CCT**: única combinación ordenada de más de 2 caracteres encontrada frecuentemente.
- **CT**: es la combinación de 2 caracteres ordenados más frecuente de todas las clases, y por tanto en general.
- **TG**: combinación también bastante frecuente en todas las clases.
- **CC**: la combinación de 2 caracteres iguales más frecuente en general, lo cual tiene sentido por que el caracter **C** también lo es.

A continuación, se repite el procedimiento introduciendo a la función **GSP** un soporte mayor, **0.8**.


### GSP con soporte 0.8

In [ ]:
results2 = GSP(datos['Split']).search(0.8)
resultEI2 = GSP(datos['Split'][datos['Clase']=='EI']).search(0.8)
resultIE2 = GSP(datos['Split'][datos['Clase']=='IE']).search(0.8)
resultN2 = GSP(datos['Split'][datos['Clase']=='N']).search(0.8)

In [ ]:
print('Patrones secuenciales generales:\n',results2)
print('\n\nPatrones secuenciales clase EI:\n',resultEI2)
print('\n\nPatrones secuenciales clase IE:\n',resultIE2)
print('\n\nPatrones secuenciales clase N:\n',resultN2)

Patrones secuenciales generales:
 [{('C',): 2424, ('A',): 2351, ('G',): 2370, ('T',): 2346}]


Patrones secuenciales clase EI:
 [{('C',): 579, ('G',): 572}]


Patrones secuenciales clase IE:
 [{('C',): 614, ('T',): 595}]


Patrones secuenciales clase N:
 [{('C',): 1231, ('A',): 1269, ('G',): 1273}]


Con un soporte de **0.8**, solo se encuentran con frecuencia caracteres individuales. Se prueba a bajar el soporte hasta encontrar uno donde empiecen a aparecer combinaciones de mayor longitud. 

### GSP con soporte 0.5

In [ ]:
results3 = GSP(datos['Split']).search(0.5)
resultEI3 = GSP(datos['Split'][datos['Clase']=='EI']).search(0.5)
resultIE3 = GSP(datos['Split'][datos['Clase']=='IE']).search(0.5)
resultN3 = GSP(datos['Split'][datos['Clase']=='N']).search(0.5)

In [ ]:
print('Patrones secuenciales generales:\n',results3)
print('\n\nPatrones secuenciales clase EI:\n',resultEI3)
print('\n\nPatrones secuenciales clase IE:\n',resultIE3)
print('\n\nPatrones secuenciales clase N:\n',resultN3)

Patrones secuenciales generales:
 [{('C',): 2424, ('A',): 2351, ('G',): 2370, ('T',): 2346}]


Patrones secuenciales clase EI:
 [{('C',): 579, ('A',): 534, ('G',): 572, ('T',): 541}]


Patrones secuenciales clase IE:
 [{('T',): 595, ('C',): 614, ('A',): 548, ('G',): 525}, {('C', 'T'): 370}]


Patrones secuenciales clase N:
 [{('A',): 1269, ('C',): 1231, ('G',): 1273, ('T',): 1210}]


Con **0.5** de soporte, aparece la combinacion ordenada **'CT'** en la clase **IE** como única con frecuencia suficiente de longitud 2 o más. Se reduce el soporte a continuación para hallar un buen valor de este.

### GSP con soporte 0.4

In [ ]:
results4 = GSP(datos['Split']).search(0.4)
resultEI4 = GSP(datos['Split'][datos['Clase']=='EI']).search(0.4)
resultIE4 = GSP(datos['Split'][datos['Clase']=='IE']).search(0.4)
resultN4 = GSP(datos['Split'][datos['Clase']=='N']).search(0.4)

In [ ]:
print('Patrones secuenciales generales:\n',results4)
print('\n\nPatrones secuenciales clase EI:\n',resultEI4)
print('\n\nPatrones secuenciales clase IE:\n',resultIE4)
print('\n\nPatrones secuenciales clase N:\n',resultN4)

Patrones secuenciales generales:
 [{('C',): 2424, ('A',): 2351, ('G',): 2370, ('T',): 2346}, {('C', 'T'): 1271, ('T', 'G'): 1166}]


Patrones secuenciales clase EI:
 [{('C',): 579, ('A',): 534, ('G',): 572, ('T',): 541}, {('C', 'T'): 294, ('G', 'C'): 281, ('T', 'G'): 282}]


Patrones secuenciales clase IE:
 [{('T',): 595, ('C',): 614, ('A',): 548, ('G',): 525}, {('C', 'C'): 284, ('C', 'T'): 370, ('T', 'C'): 285}]


Patrones secuenciales clase N:
 [{('C',): 1231, ('A',): 1269, ('G',): 1273, ('T',): 1210}]


Con **0.4** de soporte, empiezan a aparecer más combinaciones de longitud 2. Este, podría ser un buen soporte para estudiar las frecuencias de este tipo de combinaciones, ya que la librería detecta varias y nos aseguramos que son solo las más importantes. Igualmente, esto solo sería aplicable a las clases **EI** y **IE**, ya que todavía no detecta estas combinaciones en la clase **N**, por lo que para esta habría que bajar un poco el valor del soporte para esta clase.
- **CT** y **TG**: aparecen generalmente con frecuencia.
- **GC**: aparece con frecuencia en la clase **EI**.
- **TC** y **CC**: aparecen con frecuencia en la clase **IE**.

### GSP con soporte 0.1

In [ ]:
results5 = GSP(datos['Split']).search(0.15)
resultEI5 = GSP(datos['Split'][datos['Clase']=='EI']).search(0.15)
resultIE5 = GSP(datos['Split'][datos['Clase']=='IE']).search(0.15)
resultN5 = GSP(datos['Split'][datos['Clase']=='N']).search(0.15)

In [ ]:
print('Patrones secuenciales generales:\n',results5)
print('\n\nPatrones secuenciales clase EI:\n',resultEI5)
print('\n\nPatrones secuenciales clase IE:\n',resultIE5)
print('\n\nPatrones secuenciales clase N:\n',resultN5)

Patrones secuenciales generales:
 [{('C',): 2424, ('A',): 2351, ('T',): 2346, ('G',): 2370}, {('A', 'A'): 781, ('A', 'C'): 863, ('A', 'G'): 1036, ('A', 'T'): 800, ('C', 'A'): 1084, ('C', 'C'): 996, ('C', 'G'): 470, ('C', 'T'): 1271, ('G', 'A'): 1024, ('G', 'C'): 1041, ('G', 'G'): 921, ('G', 'T'): 675, ('T', 'A'): 560, ('T', 'C'): 965, ('T', 'G'): 1166, ('T', 'T'): 745}, {('C', 'T', 'G'): 438}]


Patrones secuenciales clase EI:
 [{('C',): 579, ('A',): 534, ('G',): 572, ('T',): 541}, {('A', 'A'): 162, ('A', 'C'): 192, ('A', 'G'): 259, ('A', 'T'): 173, ('C', 'A'): 238, ('C', 'C'): 229, ('C', 'G'): 144, ('C', 'T'): 294, ('G', 'A'): 269, ('G', 'G'): 242, ('G', 'C'): 281, ('G', 'T'): 135, ('T', 'C'): 217, ('T', 'T'): 142, ('T', 'G'): 282}, {('C', 'T', 'G'): 109, ('G', 'C', 'T'): 106, ('T', 'G', 'G'): 102}]


Patrones secuenciales clase IE:
 [{('T',): 595, ('C',): 614, ('A',): 548, ('G',): 525}, {('A', 'A'): 186, ('A', 'C'): 237, ('A', 'T'): 184, ('C', 'A'): 268, ('A', 'G'): 186, ('C', 'C'): 

Con **0.15** de soporte, empiezan a aparecer más combinaciones de longitud 3. Este, podría ser un buen soporte para estudiar las frecuencias de este tipo de combinaciones, ya que la librería detecta varias y nos aseguramos que son solo las más importantes. Igualmente, esto solo sería aplicable a las clases **EI** y **IE**, ya que todavía no detecta estas combinaciones en la clase **N**, por lo que para esta habría que bajar un poco el valor del soporte para esta clase. Además, se ha necesitado un valor muy bajo de soporte, por lo que quizás estos patrones no tengan relevancia.
- **CTG**: aparece generalmente con frecuencia, aunque también y con mayor frecuencia **CT** y **TG**, por lo que puede ser resultado de esto.
- **GCT** y **TGG**: aparece con frecuencia en la clase **EI**.
- **CTC** y **CTG**: aparecen con frecuencia en la clase **IE**.

## Conclusiones
Se ha conseguido con éxito un método óptimo para encontrar patrones secuenciales a partir de un dataset, el cual es replicable y generalizable para otros casos.

Se han encontrado combinaciones ordenadas frecuentes principalmente de longitud 2, que pueden estudiarse en mayor profundidad biológica para entender el por qué de sus altas frecuencias. Además, se puede hacer lo mismo con los caracteres individuales, o incluso con combinaciones de longitud 3, aunque estas sean menos frecuentes.

También, se ha observado que la clase **N** requiere de soportes más bajos que las otras clases para encontrar patrones secuenciales, por lo que se puede deducir que sus caracteres están más repartidos y ordenados uniformemente.